In [84]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt


In [229]:
tickers = ['AAPL', 'C', 'AMZN', 'DIS', 'NVDA', 'UNH', 'MSFT']
etfs = ['SPY', 'IWM', 'DIA']

date_end = dt.datetime.now()
date_start = date_end - dt.timedelta(int(10*365))

port_data = yf.download(tickers + etfs, period = '10y')['Adj Close']

risk_data = yf.download(tickers + etfs, period = '10y')['Adj Close']

p_returns = port_data.pct_change()

ann_ret = p_returns[-252:]

ann_vol = (p_returns[-63:].std()) * (252 ** (1/2))

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed


In [230]:
port_data.head()

,AAPL,AMZN,C,DIA,DIS,IWM,MSFT,NVDA,SPY,UNH
Date,,,,,,,,,,
2013-10-07,15.203006,15.5015,38.710709,120.517548,58.718300,92.479668,27.885904,3.657556,139.549759,61.915344
2013-10-08,14.990745,15.1615,38.300938,119.328552,58.181942,90.984497,27.643061,3.627057,137.924469,61.323395
2013-10-09,15.166853,14.9115,38.525902,119.530777,57.809208,90.617279,27.693310,3.570750,138.024521,61.237591
2013-10-10,15.261919,15.2585,39.586468,122.102852,59.618313,92.803162,28.271114,3.610634,141.000046,63.468140
2013-10-11,15.360725,15.5445,39.546303,123.081657,60.191040,94.149635,28.580965,3.580135,141.908554,63.716942


In [231]:
result = pd.DataFrame(index = tickers)

result['weight'] = 100 / len(tickers)

result['ann_vol'] = ann_vol

result

,weight,ann_vol
AAPL,14.285714,0.215355
C,14.285714,0.196506
AMZN,14.285714,0.328352
DIS,14.285714,0.244820
NVDA,14.285714,0.398036
UNH,14.285714,0.224035
MSFT,14.285714,0.227751


In [232]:
port_data.cov()

,AAPL,AMZN,C,DIA,DIS,IWM,MSFT,NVDA,SPY,UNH
AAPL,3008.290679,2417.412659,154.312758,3778.563236,784.156479,1815.498855,5327.044099,5284.845752,5121.651441,7734.523280
AMZN,2417.412659,2660.039436,261.935259,3431.523218,1144.449593,1754.388841,4583.618925,3994.353052,4518.206704,6349.370735
C,154.312758,261.935259,93.753784,378.877906,203.534260,238.134121,346.180939,234.070699,438.762666,510.505730
DIA,3778.563236,3431.523218,378.877906,5347.125222,1329.123931,2621.509747,6958.613393,6551.755745,6957.111645,10375.034561
DIS,784.156479,1144.449593,203.534260,1329.123931,852.286054,818.491979,1593.642919,994.358792,1721.144162,1982.960095
IWM,1815.498855,1754.388841,238.134121,2621.509747,818.491979,1417.399714,3345.863296,3101.364015,3434.444949,4874.847993
MSFT,5327.044099,4583.618925,346.180939,6958.613393,1593.642919,3345.863296,9716.426436,9452.091090,9344.787576,13864.833030
NVDA,5284.845752,3994.353052,234.070699,6551.755745,994.358792,3101.364015,9452.091090,10838.545263,8975.398779,13227.846953
SPY,5121.651441,4518.206704,438.762666,6957.111645,1721.144162,3434.444949,9344.787576,8975.398779,9224.968739,13637.346058
UNH,7734.523280,6349.370735,510.505730,10375.034561,1982.960095,4874.847993,13864.833030,13227.846953,13637.346058,21595.236691


In [233]:
# result['beta_spy'] = ann_ret.cov()['SPY'] / ann_ret['SPY'].var()
# result['beta_iwm'] = ann_ret.cov()['IWM'] / ann_ret['IWM'].var()
# result['beta_dia'] = ann_ret.cov()['DIA'] / ann_ret['DIA'].var()

In [234]:
for etf in etfs:
    result['beta_' +etf.lower()] = ann_ret.cov()[etf] / ann_ret[etf].var()

In [235]:
result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia
AAPL,14.285714,0.215355,1.327643,0.833916,1.345551
C,14.285714,0.196506,1.060959,0.921424,1.288150
AMZN,14.285714,0.328352,1.554364,0.994008,1.358297
DIS,14.285714,0.244820,1.229411,0.954847,1.407098
NVDA,14.285714,0.398036,2.151440,1.300109,1.881497
UNH,14.285714,0.224035,0.389200,0.221559,0.737295
MSFT,14.285714,0.227751,1.380858,0.759801,1.325093


In [236]:
drawdown = (port_data / port_data.rolling(window=252).max()) - 1

In [237]:
result['avg drawdown'] = drawdown.mean()
result['max_drawdown'] = drawdown.min()

result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia,avg drawdown,max_drawdown
AAPL,14.285714,0.215355,1.327643,0.833916,1.345551,-0.090380,-0.385159
C,14.285714,0.196506,1.060959,0.921424,1.288150,-0.154294,-0.565076
AMZN,14.285714,0.328352,1.554364,0.994008,1.358297,-0.114334,-0.533882
DIS,14.285714,0.244820,1.229411,0.954847,1.407098,-0.144185,-0.505998
NVDA,14.285714,0.398036,2.151440,1.300109,1.881497,-0.142095,-0.663351
UNH,14.285714,0.224035,0.389200,0.221559,0.737295,-0.052182,-0.358999
MSFT,14.285714,0.227751,1.380858,0.759801,1.325093,-0.062173,-0.371485


In [238]:
tot_ret = (port_data.iloc[-1] / port_data.iloc[0]) - 1
an_returns = ((1 + tot_ret) ** (1 / 10)) 

In [239]:
result['Ann Tot Ret'] = an_returns

result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia,avg drawdown,max_drawdown,Ann Tot Ret
AAPL,14.285714,0.215355,1.327643,0.833916,1.345551,-0.090380,-0.385159,1.275784
C,14.285714,0.196506,1.060959,0.921424,1.288150,-0.154294,-0.565076,1.002804
AMZN,14.285714,0.328352,1.554364,0.994008,1.358297,-0.114334,-0.533882,1.234079
DIS,14.285714,0.244820,1.229411,0.954847,1.407098,-0.144185,-0.505998,1.030531
NVDA,14.285714,0.398036,2.151440,1.300109,1.881497,-0.142095,-0.663351,1.614606
UNH,14.285714,0.224035,0.389200,0.221559,0.737295,-0.052182,-0.358999,1.234895
MSFT,14.285714,0.227751,1.380858,0.759801,1.325093,-0.062173,-0.371485,1.275952


In [270]:
riskdf = pd.DataFrame(index = etfs)

riskdf['corr'] = p_returns.corrwith(port_data)

covar = p_returns.cov(port_data)
riskdf['covar'] = covar

riskdf['track_err'] = np.sqrt(covar.diagonal())

riskdf['sharpe ratio'] = (tot_ret - 0.02) / ann_vol

riskdf


TypeError: an integer is required